In [33]:
%load_ext autoreload
%autoreload 2

from plotting_funcs import*
from pre_process import *
from get_rossby_radius import *
import pandas as pd
import seawater as sw
from scipy.interpolate import interp1d

import warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


__Fourth approach__
Idea: do everything on arrays and discard the dataset structure. Similar to the third approach.
Code and logic inspired by Ann Kristin Sperrevik, Kai H. Christensen and Victor C. M. de Aguiar.

In [34]:
def compute_N2(rho_, z_, rho_0=1000.0):
    '''
    --!Function stolen from Ann Kristin Sperrevik!--
    Return the stratification frequency

    Parameters
    ----------
    rho : array_like
        density [kg/m^3]
    z : array_like
        depths [m] (positive upward)

    Returns
    -------
    N2 : array_like
        Stratification frequency [1/s], where the vertical dimension (the
        first dimension) is one less than the input arrays.
    '''
    rho = np.asarray(rho_)
    z = np.asarray(z_)

    assert rho.shape == z.shape, 'rho and z must have the same shape.'
    r_z = np.diff(rho, axis=0) / np.diff(z, axis=0)

    buoy_freq_sq =  -(9.8 / rho_0) * r_z

    return buoy_freq_sq

def deformation_radius(turb, month, day):
    '''
    Opens a requested dataset and computes Rossby deformation radius.

    Parameters
    ----------
    turb : boolean
        True: experiment with turbines, False: reference experiment
    month : str
        '02', '03', etc.
    day : int
        day of month

    Returns
    -------
    R1 : array_like
        Rossby deformation radius (first baroclinic mode) [m]
        given over whole model domain with dimensions (lon, lat)
    
    '''

    # Identifying which dataset to open
    if turb==True:
        base_path = f'/lustre/storeB/project/nwp/havvind/hav/results/experimeDt/EXP-{month}/norkyst_avg_'
    else:
        base_path = f'/lustre/storeB/project/nwp/havvind/hav/results/reference/REF-{month}/norkyst_avg_'

    if day < 10:
        d = f'000{day}'
    else:
        d = f'00{day}'

    path = base_path+d+'.nc'

    ds = xroms.open_netcdf(path)
    ds, grid = xroms.roms_dataset(ds, include_cell_volume=True)

    # Fetching the variables needed
    lat = ds.lat_rho.values  # latitudes of whole domain
    lon = ds.lon_rho.values  # longitudes of whole domain
    rho = ds.rho.values  # Density
    f = ds.f.values  # Coriolis parameter
    time = ds.ocean_time  # time stamp
    mask = ds.mask_rho.values  # Land mask (==0 for land)
    z_r = ds.z_rho.values  # Depth of rho points
    h = ds.h.values  # Bathymetry
    hc = ds.hc.values  # critical depth
    Cs_w = ds.Cs_w.values  # Stretching w points
    s_w = ds.s_w.values  # s-coordinate at w points
    Cs_r = ds.Cs_r.values  # Stretching rho points
    s_r = ds.s_rho.values  # S-coord at rho points
    zeta = ds.zeta.values  # free-surface elevation

    # Dimensions
    Dx = rho.shape[3]  # x
    Dy = rho.shape[2]  # y
    Dt = rho.shape[0]  # time
    Dz = rho.shape[1]  # depth

    N2 = np.full((rho.shape[1], rho.shape[2], rho.shape[3]), np.nan)
    R1 = np.full((rho.shape[2], rho.shape[3]), np.nan)

    # Loop through grid points and interpolate to z levels
    for j in range(Dy):
        for i in range(Dx):
            if mask[j, i] == 0:  # skipping grid points on land
                continue
            z = np.zeros(Dz+1)

            for k in range(Dz+1):
                S = (hc * s_w[k] + h[j, i] * Cs_w[k]) / (hc + h[j, i])
                z[k] = zeta[0, j, i] + (zeta[0, j, i] + h[j, i]) * S


            rho_interpolated = interp1d(
                z_r[:, j, i],  # Original density levels
                rho[0, :, j, i],  # Density values at original levels
                bounds_error=False,
                fill_value="extrapolate"
            )(z)  # Evaluate at new depth levels


            N2[:, j, i] = compute_N2(rho_interpolated[0, :, j, i], z)
            # Avoid negative values
            N2[N2 < 0] = 0.0000001

            # Calculate buoyancy frequency N
            N = np.sqrt(N2)

            z_mid = (z[:-1] + z[1:]) / 2  # Midpoints for the integration

            # Integrate N over depth
            N_int = np.trapz(N, z_mid, axis=0)

            R1[j, i] = 1/(np.abs(f)*np.pi) * N_int

    return lat, lon, R1

In [ ]:
lat, lon, R1 = deformation_radius(turb=False, month='05', day=31)